Завдання.
Проведіть класифікацію на основі датасету UCI ML Breast Cancer Wisconsin, використовуючи моделі бустингу, беггінгу та стекингу.

Змініть їх такими параметрами:

у StackingClassifier як базові алгоритми встановіть LogisticRegression та DecisionTreeClassifier,
як фінальну модель - SVC (Support Vector Classification.);


у BaggingClassifier як базовий алгоритм встановіть DecisionTreeClassifier;

у AdaBoostClassifier як базовий алгоритм встановіть LogisticRegression;

у XGBClassifier як базовий алгоритм (booster) встановіть 'gbtree'.

У зведеній таблиці порівняйте результати наведених у коді класифікацій із класифікаціями, що отримані вами.

In [2]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Завантаження датасету
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
column_names = ['ID', 'Diagnosis', 'Mean Radius', 'Mean Texture', 'Mean Perimeter', 'Mean Area', 'Mean Smoothness',
                'Mean Compactness', 'Mean Concavity', 'Mean Concave Points', 'Mean Symmetry', 'Mean Fractal Dimension',
                'SE Radius', 'SE Texture', 'SE Perimeter', 'SE Area', 'SE Smoothness', 'SE Compactness', 'SE Concavity',
                'SE Concave Points', 'SE Symmetry', 'SE Fractal Dimension', 'Worst Radius', 'Worst Texture',
                'Worst Perimeter', 'Worst Area', 'Worst Smoothness', 'Worst Compactness', 'Worst Concavity',
                'Worst Concave Points', 'Worst Symmetry', 'Worst Fractal Dimension']
data = pd.read_csv(url, header=None, names=column_names)

# Розділення на ознаки та цільову змінну
X = data.iloc[:, 2:]
y = data['Diagnosis']

# Розбиття на тренувальний та тестовий набори даних
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data


,ID,Diagnosis,Mean Radius,Mean Texture,Mean Perimeter,Mean Area,Mean Smoothness,Mean Compactness,Mean Concavity,Mean Concave Points,...,Worst Radius,Worst Texture,Worst Perimeter,Worst Area,Worst Smoothness,Worst Compactness,Worst Concavity,Worst Concave Points,Worst Symmetry,Worst Fractal Dimension
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [4]:
from sklearn.preprocessing import StandardScaler

# Масштабування даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Беггінг (Bagging) з DecisionTreeClassifier як базовим алгоритмом
bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=42)
bagging.fit(X_train, y_train)
bagging_pred = bagging.predict(X_test)
bagging_accuracy = accuracy_score(y_test, bagging_pred)

# Вивід точності Bagging
print("Точність классификатора Bagging:", bagging_accuracy)

Точність классификатора Bagging: 0.956140350877193


In [6]:
# AdaBoost з LogisticRegression як базовим алгоритмом
adaboost = AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear'), random_state=42)
adaboost.fit(X_train, y_train)
adaboost_pred = adaboost.predict(X_test)
adaboost_accuracy = accuracy_score(y_test, adaboost_pred)

# Вивід точності AdaBoost
print("Точність классификатора AdaBoost:", adaboost_accuracy)

Точність классификатора AdaBoost: 0.956140350877193


In [13]:
# Stacking з LogisticRegression та DecisionTreeClassifier як базовими алгоритмами та SVC як фінальною моделлю
estimators = [
    ('lr', LogisticRegression(solver='liblinear', max_iter=1000)),
    ('dt', DecisionTreeClassifier())
]
stacking = StackingClassifier(estimators=estimators, final_estimator=SVC())
stacking.fit(X_train_scaled, y_train)
stacking_pred = stacking.predict(X_test_scaled)
stacking_accuracy = accuracy_score(y_test, stacking_pred)

# Вивід точності Stacking
print("Точність классификатора Stacking:", stacking_accuracy)

Точність классификатора Stacking: 0.9824561403508771


In [14]:
from sklearn.preprocessing import LabelEncoder

# Створення інстанції LabelEncoder
label_encoder = LabelEncoder()

# Перетворення рядкових міток на числові значення
y_train_encoded = label_encoder.fit_transform(y_train)

# XGBoost з базовим алгоритмом 'gbtree'
xgboost = XGBClassifier(booster='gbtree', random_state=42)
xgboost.fit(X_train, y_train_encoded)
xgboost_pred = xgboost.predict(X_test)

# Перетворення числових міток назад на рядкові
y_pred_decoded = label_encoder.inverse_transform(xgboost_pred)

xgboost_accuracy = accuracy_score(y_test, y_pred_decoded)

# Вивід точності XGBoost
print("Точність классификатора XGBoost:", xgboost_accuracy)

Точність классификатора XGBoost: 0.956140350877193


In [17]:
results_from_lesson = [0.9122807017, 0.9415204678, 0.906432748, 0.8538011695]

results = pd.DataFrame({
    'Model': ['Bagging', 'AdaBoost', 'Stacking', 'XGBoost'],
    'Accuracy (From Lesson)': results_from_lesson,
    'Accuracy (Your Results)': [bagging_accuracy, adaboost_accuracy, stacking_accuracy, xgboost_accuracy]
})
print(results)


      Model  Accuracy (From Lesson)  Accuracy (Your Results)
0   Bagging                0.912281                 0.956140
1  AdaBoost                0.941520                 0.956140
2  Stacking                0.906433                 0.982456
3   XGBoost                0.853801                 0.956140
